### Quiero crear un mapa de España donde ficguren la totalidad de los festivales de los diferentes estilos. Cada capa será un estilo (agrupado de música) de música

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import unidecode
import json
import numpy as np
from datetime import datetime

import folium # pip install folium
from folium import plugins

from geopy.geocoders import Nominatim

In [4]:
# !pip install folium
# !pip install geopy

Importo un csv con la lista de las url de donde tengo que acceder y traerme los adtos necesarios para la creación del mapa Folium de España 

In [12]:
lista_url = pd.read_csv(r"..\data\datos_creacion_mapa\lista_url.csv", index_col="Unnamed: 0")
lista_url = list(lista_url["url"])
lista_url

['https://fanmusicfest.com//content/lovin-ibiza-festival-2023',
 'https://fanmusicfest.com//content/ims-dalt-vila-2023',
 'https://fanmusicfest.com//content/darkmad-2023',
 'https://fanmusicfest.com//content/tv-hardcore-festival-2023',
 'https://fanmusicfest.com//content/valencia-flow-fest-2023',
 'https://fanmusicfest.com//content/ritmo-festival-2023',
 'https://fanmusicfest.com//content/festival-dias-de-campo-2023',
 'https://fanmusicfest.com//content/molan-los-90-zaragoza-2023',
 'https://fanmusicfest.com//content/love-90s-valencia-2023',
 'https://fanmusicfest.com//content/homenaje-la-ruta-90s-2mil-benidorm-2023',
 'https://fanmusicfest.com//content/sonar-2023',
 'https://fanmusicfest.com//content/offsonar-2023',
 'https://fanmusicfest.com//content/o-son-do-camino-2023',
 'https://fanmusicfest.com//content/offweek-festival-2023',
 'https://fanmusicfest.com//content/animal-sound-festival-2023',
 'https://fanmusicfest.com//content/love-90s-madrid-2023',
 'https://fanmusicfest.com//co

Código para extraer de las diferentes urls las imágenes de los carteles, pensado para crear un panel de mando en Power BI donde se pueda ver una especie de ficha de cada festival

In [7]:
# código para es

# for index, row in df6.iterrows():

#     filename = f'{row["nombre"]}.jpg'
#     response = requests.get(row["url"])
#     open(filename, 'wb').write(response.content)

Creo un diccionario donde iré recopilando los datos que extraiga de la web

In [13]:
dicc_mapa_Folium = {
                    "festival": [],
                    "local": [],
                    "direccion": [],
                    "ciudad": [],
                    "provincia": [],
                    "latitud": [],
                    "longitud": [],
                    "fecha_inicio": [],
                    "fecha_fin": [],
                    "duracion": [],
                    "estilo": []
                   }

Función para extraer los datos de la web a través de una lista de URLs

In [15]:
def get_data(lista_URL):
    ''' 
    Función que conecta con la página web para extraer los datos de cada uno de los festivales y la almacena en un diccionario

    input: Recibe una lista (lst) de las URLs para cada estilo de música
    output: Retorna una diccionario (dict) con la información recogida de cada uno de los festivales
    
    '''
    for index, row in df6.iterrows():

        try:
            response = requests.get(row["url"])
            html = response.content
            soup = bs(html, "lxml")

            geoJson = soup.find_all("script", {"type": "application/ld+json"})

            nombre = row["nombre"]
            local = json.loads(geoJson[0].getText())["@graph"][0]["location"]["name"]
            direccion = json.loads(geoJson[0].getText())["@graph"][0]["location"]["address"]["streetAddress"]
            ciudad = json.loads(geoJson[0].getText())["@graph"][0]["location"]["address"]["addressLocality"]
            provincia= json.loads(geoJson[0].getText())["@graph"][0]["location"]["address"]["addressRegion"]
            latitud = json.loads(geoJson[0].getText())["@graph"][0]["location"]["geo"]["latitude"]
            longitude = json.loads(geoJson[0].getText())["@graph"][0]["location"]["geo"]["longitude"]
            dia_inicio = pd.to_datetime(json.loads(geoJson[0].getText())["@graph"][0]["startDate"])
            dia_fin = pd.to_datetime(json.loads(geoJson[0].getText())["@graph"][0]["endDate"])
            duracion = int(str((dia_fin - dia_inicio)).split(" ")[0])+1
            estilo = row["estilo"]
            
            dicc_mapa_Folium["festival"].append(nombre)
            dicc_mapa_Folium["local"].append(local)
            dicc_mapa_Folium["direccion"].append(direccion)
            dicc_mapa_Folium["ciudad"].append(ciudad)
            dicc_mapa_Folium["provincia"].append(provincia)
            dicc_mapa_Folium["latitud"].append(latitud)
            dicc_mapa_Folium["longitud"].append(longitude)
            dicc_mapa_Folium["fecha_inicio"].append(dia_inicio)
            dicc_mapa_Folium["fecha_fin"].append(dia_fin)
            dicc_mapa_Folium["duracion"].append(duracion)
            dicc_mapa_Folium["estilo"].append(estilo)
                    
        except:
            pass
 
    return dicc_mapa_Folium

Extraigo los datos de la web

In [16]:
get_data(lista_url)

{'festival': ['Lovin Ibiza Festival',
  'IMS Dalt Vila',
  'DarkMAD',
  'Tv Hardcore Festival',
  'Valencia Flow Fest',
  'Ritmo Festival',
  'Festival Días de Campo',
  'Molan los 90 Zaragoza',
  'Love the 90’s Valencia',
  "Homenaje a la Ruta 90's & 2MIL Benidorm",
  'Sónar',
  'OFFSónar',
  'O Son do Camiño',
  'OffWeek Festival',
  'Animal Sound Festival',
  'Love the 90’s Madrid',
  'A Summer Story',
  "Love the Tuenti's",
  'Ultra Beach Costa del Sol',
  'Holika Festival',
  'Maudes Festival',
  'Weekend Beach Festival',
  "Homenaje a la Ruta 90's & 2MIL Valencia",
  'FIB',
  'Fan Futura Fest',
  'Monegros Desert Festival',
  'Loona Summer Festival',
  'Arenal Sound',
  'Dreambeach Festival',
  'Medusa Sun Beach Festival',
  'Aquasella',
  'The End of the World Festival',
  'Love the 90’s Alicante',
  'Parallel Festival',
  'Extremúsika',
  'WAN Festival',
  'Palma On Season Fest',
  'AbroadFest',
  'Snowdaze',
  'Iboga Rave Madrid',
  'Makinando Festival',
  'Primavera Festival 

In [18]:
for key, value in dicc_mapa_Folium.items():
    print(key, len(value))

festival 545
local 545
direccion 545
ciudad 545
provincia 545
latitud 545
longitud 545
fecha_inicio 545
fecha_fin 545
duracion 545
estilo 545


A partir del diccionario creo el dataframe del que extraeré los puntos de latitud y longitud para el mapa

In [19]:
df_mapa_Folium = pd.DataFrame(dicc_mapa_Folium)

In [20]:
df_mapa_Folium["estilo"].value_counts()

indie, pop             168
rock, metal            144
fusion, world music    108
musica urbana           78
electronica             47
Name: estilo, dtype: int64

In [ ]:
# df_mapa_Folium.to_csv("df_mapa_folium.csv")

In [21]:
df_folium = pd.read_csv(r"..\data\datos_creacion_mapa\df_mapa_folium.csv", index_col= "Unnamed: 0")
df_folium

,festival,local,direccion,ciudad,provincia,latitude,longitude,fecha_inicio,fecha_fin,duracion,estilo,dia_mes
0,Lovin Ibiza Festival,Lío Ibiza,"Passeig Joan Carles I, 1",Eivissa,Islas Baleares,38.913852,1.443275,2023-04-28,2023-04-30,3,electronica,28-04
1,IMS Dalt Vila,"['Baluarte de Santa Lucía', 'Dalt Vila']","Carrer Baluarte Santa Lucía, 1",Ibiza,Islas Baleares,38.908434,1.438789,2023-04-28,2023-04-28,1,electronica,28-04
2,DarkMAD,Autocine,"C. de la Isla de Java, 2",Madrid,Madrid,40.486224,-3.677802,2023-04-29,2023-04-30,2,electronica,29-04
3,Tv Hardcore Festival,Family Club,"Carretera Madrid-Ciudad Real, Km 96",Sonseca,Toledo,39.675929,-3.960813,2023-04-29,2023-04-29,1,electronica,29-04
4,Valencia Flow Fest,La 3 park,"Carrera del Riu, 399",Pinedo,Valencia,39.410957,-0.334871,2023-05-06,2023-05-07,2,electronica,06-05
...,...,...,...,...,...,...,...,...,...,...,...,...
540,Valladolid Blues,Teatro Carrión,"C. Montero Calvo, 2",Valladolid,Valladolid,41.649996,-4.728177,2023-02-10,2023-03-10,29,"rock, metal",10-02
541,Carrion Rock Festival,Sala Infinity,"Calle Extremadura, 5",Palencia,Palencia,42.006608,-4.519082,2023-02-11,2023-02-11,1,"rock, metal",11-02
542,Wáchina Wáchina Fest,16 Toneladas,Ricardo Micó 3,Valencia,Valencia,39.481255,-0.387633,2023-02-18,2023-02-18,1,"rock, metal",18-02
543,Black Sound Fest,Teatro Imperial,"C. Doña Consuelo Torres, 6",Don Benito,Badajoz,38.953452,-5.860906,2023-02-04,2023-02-25,22,"rock, metal",04-02


Corrijo un error en la latitud, longitud de uno de los festivales

In [22]:
# 39.6759291,-3.9608126,15

df_mapa_Folium[df_mapa_Folium["festival"] == "Tv Hardcore Festival"]
df_mapa_Folium = df_mapa_Folium.rename(columns={"latitud": "latitude", "longitud": "longitude"})
for index, row in df_mapa_Folium.iterrows():
    if row["festival"] == "Tv Hardcore Festival":
        df_mapa_Folium.at[index, "latitude"] = "39.6759291"
        df_mapa_Folium.at[index, "longitude"] = "-3.9608126"

Añado una columna para saber el dia y mes del festival

In [ ]:
df_mapa_Folium["dia_mes"] = df_mapa_Folium["fecha_inicio"].dt.strftime("%d-%m")

Verifico los festivales que se celebran en la província de Soria

In [63]:
df_folium[df_folium["provincia"]== "Soria"]

,festival,local,direccion,ciudad,provincia,latitude,longitude,fecha_inicio,fecha_fin,duracion,estilo,dia_mes
360,Covaleda Fest,Campamento Juvenil Raso de Nava,"Paraje Raso de la Nava, s/n",Covaleda,Soria,41.937690,-2.909918,2023-07-13,2023-07-15,3,musica urbana,13-07
485,Motorbeach Festival,Playa Amogable,NaN,Vinuesa,Soria,41.858037,-2.807406,2023-07-13,2023-07-16,4,"rock, metal",13-07
493,Enclave de Agua,Paraje natural Soto Playa,"Paseo de San Prudencio, 14",Soria,Soria,41.762797,-2.454908,2023-07-27,2023-07-29,3,"rock, metal",27-07


Creo el mapa Folium con todos los festivales y según el estilo de música una capa del mismo mapa

In [24]:
# Crear un objeto Map de Folium
festival_map = folium.Map(location=[40.416719, -3.684016], zoom_start=7)

minimap_center = ["28.327749","-17.1117421"]

minimap = folium.plugins.MiniMap(toggle_display=True, position='bottomleft', width=150, height=150, zoom_level_fixed=10, center_fixed=minimap_center)
minimap.add_to(festival_map)

# Definir un diccionario de estilos y sus iconos correspondientes
estilo_icono_dict = {
                    "electronica": ["headphones", "green"],
                    "indie, pop": ["hand", "blue"],
                    "fusion, world music": ["music", "red"],
                    "musica urbana" : ["eject", "orange"],
                    "rock, metal": ["guitar", "black"]
                    }

# icon_image='ruta/al/icono.png'

# Crear un FeatureGroup para cada estilo
for estilo, icono in estilo_icono_dict.items():
    fg = folium.FeatureGroup(name=estilo)  # Crear un FeatureGroup con el nombre del estilo
    for index, row in df_folium.iterrows():
        if row["estilo"] == estilo:
            # Crear un marcador para cada registro con el estilo correspondiente y su icono personalizado
            folium.Marker(location=[row['latitude'], row['longitude']],
                          popup=row["local"],
                          tooltip=row['festival'],
                          icon=folium.Icon(color=icono[1], icon=icono[0], prefix='fa')).add_to(fg)
    fg.add_to(festival_map)  # Añadir el FeatureGroup al mapa

# Añadir el control de capas al mapa
folium.LayerControl().add_to(festival_map)

festival_map.save(r"..\data\datos_creacion_mapa\mapa_festivales.html")

# Mostrar el mapa
festival_map

Accedo a la web de Valdelavilla en Soria para conocer los establecimientos hoteleros y restaurantes

In [25]:
response = requests.get("https://valdelavilla.es/donde-dormir/")
html = response.content
soup = bs(html, "lxml")

In [26]:
donde_dormir_valdelavilla = soup.find_all("h2", {"class": "elementor-heading-title elementor-size-default"})
alojamientos = []
for alojamiento in donde_dormir_valdelavilla[2:-3]:
    alojamientos.append(alojamiento.get_text())
alojamientos

['Los Arcos de Yanguas',
 'La Ruta de las Fuentes',
 'Montes de Alhama',
 'El Molino de Bretún',
 'Castellar de Taniñe',
 'Hotel Los Cerezos',
 'Pensión PILI',
 'Hostal Paso del Fuego',
 'Casa Rural San Millán',
 'Casa Triskelion',
 'VUT Modo Avión',
 'Casa Santa Elena',
 'El Rimero de la Quintina',
 'Albergue Turístico Tierras Altas']

In [27]:
response = requests.get("https://valdelavilla.es/donde-comer/")
html = response.content
soup = bs(html, "lxml")

In [28]:
donde_comer_valdelavilla = soup.find_all("h2", {"class": "elementor-heading-title elementor-size-default"})
restaurantes = []
for restaurante in donde_comer_valdelavilla[2:-3]:
    restaurantes.append(restaurante.get_text())
restaurantes

['El Condado del Motores',
 'Rimero de la Quintina',
 'La Fundación Vicente Marín',
 'Albergue de Yanguas',
 'Asociación Amigos de Yanguas',
 'Bar Las Piscinas',
 'Pensión Pili',
 'San Millán',
 'Buenaventura (Magaña)']